In [ ]:
import pygame
import random
import math
import neat

# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RANDOM_COLOR = (random.randint(0,255), random.randint(0,255), random.randint(0,255))

# Set the width and height of the screen
WIDTH = 800
HEIGHT = 600

# Initialize Pygame
pygame.init()

# Set the font for displaying the generation and species
font = pygame.font.Font(None, 36)

# Set up the screen
screen = pygame.display.set_mode([WIDTH, HEIGHT])

# Set the caption of the window
pygame.display.set_caption("NEAT Algorithm")

# Set up the clock for the game
clock = pygame.time.Clock()

# Define some constants for the game
GRAVITY = 0.1
JUMP_SPEED = -5

# Define the class for the player
class Player(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface([50, 50])
        self.image.fill(RANDOM_COLOR)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.velocity = 0

    def jump(self):
        self.velocity = JUMP_SPEED

    def update(self):
        self.velocity += GRAVITY
        self.rect.y += self.velocity

# Define the class for the obstacle
class Obstacle(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height):
        super().__init__()
        self.image = pygame.Surface([width, height])
        self.image.fill(BLACK)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

    def update(self):
        self.rect.x -= 5

# Define the function to create a new obstacle
def create_obstacle():
    width = random.randint(50, 100)
    height = random.randint(50, 100)
    x = WIDTH
    y = random.randint(0, HEIGHT - height)
    obstacle = Obstacle(x, y, width, height)
    return obstacle

# Define the function to evaluate the fitness of the player
def evaluate_genomes(genomes, config):
    # Create a list to hold the players and their neural networks
    players = []
    nets = []
    genomes_list = []

    # Create a list to hold the obstacles
    obstacles = pygame.sprite.Group()

    # Create the first obstacle
    obstacle = create_obstacle()
    obstacles.add(obstacle)

    # Create a list to hold the genomes and their fitness
    genome_fitness = []

    # Loop through each genome
    for genome_id, genome in genomes:
        # Create a new player and add it to the list
        player = Player(100, HEIGHT/2)
        players.append(player)

        # Create a new neural network for the genome
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)

        # Add the genome to the genomes list
        genomes_list.append(genome)

        # Set the initial fitness of the genome to 0
        genome_fitness.append(0)

    # Set the score to 0
    score = 0

    # Set the generation and species labels
    gen_label = font.render("Generation: " + str(config.generation), 1, BLACK)
    species_label = font.render("Species: " + str(config.species_id), 1, BLACK)

    # Set the flag for the game loop
    done = False

    # Start the game loop
    while not done:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

        # Get the next obstacle
        if obstacle.rect.x < -obstacle.rect.width:
            obstacle = create_obstacle()
            obstacles.add(obstacle)

        # Update the players
        for i, player in enumerate(players):
            # Get the input for the neural network
            inputs = [player.rect.y / HEIGHT,
                      (obstacle.rect.x - player.rect.x) / WIDTH,
                      obstacle.rect.y / HEIGHT,
                      obstacle.rect.height / HEIGHT]

            # Get the output from the neural network
            output = nets[i].activate(inputs)

            # Jump if the output is greater than 0.5
            if output[0] > 0.5:
                player.jump()

            # Update the player
            player.update()

            # Check for collisions with the obstacles
            if pygame.sprite.spritecollide(player, obstacles, False):
                # Decrease the fitness of the genome
                genome_fitness[i] -= 1

                # Remove the player, neural network, and genome
                players.pop(i)
                nets.pop(i)
                genomes_list.pop(i)
                genome_fitness.pop(i)

            # Check if the player has passed the obstacle
            if player.rect.x > obstacle.rect.x + obstacle.rect.width:
                # Increase the fitness of the genome
                genome_fitness[i] += 1

                # Set the score to the maximum fitness
                score = max(score, genome_fitness[i])

        # Draw the screen
        screen.fill(WHITE)
        obstacles.draw(screen)
        for player in players:
            pygame.draw.rect(screen, RANDOM_COLOR, player.rect)

        # Draw the generation and species labels
        screen.blit(gen_label, (10, 10))
        screen.blit(species_label, (10, 50))

        # Update the display
        pygame.display.flip()

        # Update the clock
        clock.tick(60)

        # Check if all the players are dead
        if len(players) == 0:
            # Set the fitness of each genome
            for i, genome in enumerate(genomes_list):
                genome.fitness = genome_fitness[i]

            # End the game loop
            done = True

    # Quit Pygame
    pygame.quit()

# Set up the NEAT configuration
config_path = 'D:/Code/Python/config-feedforward.txt' # replace with the full path to your config file
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
config.generation = 0  # Add the generation attribute to the config object
config.species_id = 0  # Add the species_id attribute to the config object

# Create the NEAT population
population = neat.Population(config)

# Add a reporter to the population
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)

# Run the NEAT algorithm
winner = population.run(evaluate_genomes)

# Display the winner
print('\nBest genome:\n{!s}'.format(winner))